# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


In [2]:
#mdl_file = 'tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl'
# mdl_file = 'tests/test-models/tests/subscript_2d_arrays/test_subscript_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1d_arrays/subscript_individually_defined_1d_arrays.mdl'
mdl_file = 'tests/pdatvensim8k_RW_v2_4.mdl'
model = pysd.read_vensim(mdl_file)

{'maintenance_type': {'preventive': 0, 'corrective': 1}, 'availability_number': {'sra14': 13, 'sra15': 14, 'sra12': 11, 'sra13': 12, 'sra10': 9, 'sra11': 10, 'sra4': 3, 'sra5': 4, 'sra6': 5, 'sra7': 6, 'sra1': 0, 'sra2': 1, 'sra3': 2, 'sra8': 7, 'sra9': 8}, 'other_availability_number': {'sra14': 13, 'sra15': 14, 'sra12': 11, 'sra13': 12, 'sra10': 9, 'sra11': 10, 'sra4': 3, 'sra5': 4, 'sra6': 5, 'sra7': 6, 'sra1': 0, 'sra2': 1, 'sra3': 2, 'sra8': 7, 'sra9': 8}, 'maintenance_level': {'onboard': 0, 'intermediate': 1, 'depot': 2}, 'support_type': {'logistics': 0, 'admin': 1, 'training': 2}, 'rank': {'r4': 3, 'r5': 4, 'r6': 5, 'r7': 6, 'r1': 0, 'r2': 1, 'r3': 2, 'r8': 7, 'r9': 8}, 'crew_type': {'contractor': 4, 'civilian': 3, 'officer': 1, 'enlisted': 0, 'warrant_officer': 2}, 'skill_type': {'other': 1, 'maintenance': 0}}


IndentationError: unexpected indent (pdatvensim8k_RW_v2_4.py, line 2284)

In [3]:
{'maintenance_type': {'preventive': 0, 'corrective': 1}, 'availability_number': {'sra14': 13, 'sra15': 14, 'sra12': 11, 'sra13': 12, 'sra10': 9, 'sra11': 10, 'sra4': 3, 'sra5': 4, 'sra6': 5, 'sra7': 6, 'sra1': 0, 'sra2': 1, 'sra3': 2, 'sra8': 7, 'sra9': 8}, 'other_availability_number': [13, 14, 11, 12, 9, 10, 3, 4, 5, 6, 0, 1, 2, 7, 8], 'maintenance_level': {'onboard': 0, 'intermediate': 1, 'depot': 2}, 'support_type': {'logistics': 0, 'admin': 1, 'training': 2}, 'rank': {'r4': 3, 'r5': 4, 'r6': 5, 'r7': 6, 'r1': 0, 'r2': 1, 'r3': 2, 'r8': 7, 'r9': 8}, 'crew_type': {'contractor': 4, 'civilian': 3, 'officer': 1, 'enlisted': 0, 'warrant_officer': 2}, 'skill_type': {'other': 1, 'maintenance': 0}}

{'availability_number': {'sra1': 0,
  'sra10': 9,
  'sra11': 10,
  'sra12': 11,
  'sra13': 12,
  'sra14': 13,
  'sra15': 14,
  'sra2': 1,
  'sra3': 2,
  'sra4': 3,
  'sra5': 4,
  'sra6': 5,
  'sra7': 6,
  'sra8': 7,
  'sra9': 8},
 'crew_type': {'civilian': 3,
  'contractor': 4,
  'enlisted': 0,
  'officer': 1,
  'warrant_officer': 2},
 'maintenance_level': {'depot': 2, 'intermediate': 1, 'onboard': 0},
 'maintenance_type': {'corrective': 1, 'preventive': 0},
 'other_availability_number': [13,
  14,
  11,
  12,
  9,
  10,
  3,
  4,
  5,
  6,
  0,
  1,
  2,
  7,
  8],
 'rank': {'r1': 0,
  'r2': 1,
  'r3': 2,
  'r4': 3,
  'r5': 4,
  'r6': 5,
  'r7': 6,
  'r8': 7,
  'r9': 8},
 'skill_type': {'maintenance': 0, 'other': 1},
 'support_type': {'admin': 1, 'logistics': 0, 'training': 2}}

In [3]:
{'maintenance_type': {'preventive': 0, 'corrective': 1}, 'availability_number': {'sra14': 13, 'sra15': 14, 'sra12': 11, 'sra13': 12, 'sra10': 9, 'sra11': 10, 'sra4': 3, 'sra5': 4, 'sra6': 5, 'sra7': 6, 'sra1': 0, 'sra2': 1, 'sra3': 2, 'sra8': 7, 'sra9': 8}, 'other_availability_number': {'sra14': 13, 'sra15': 14, 'sra12': 11, 'sra13': 12, 'sra10': 9, 'sra11': 10, 'sra4': 3, 'sra5': 4, 'sra6': 5, 'sra7': 6, 'sra1': 0, 'sra2': 1, 'sra3': 2, 'sra8': 7, 'sra9': 8}, 'maintenance_level': {'onboard': 0, 'intermediate': 1, 'depot': 2}, 'support_type': {'logistics': 0, 'admin': 1, 'training': 2}, 'rank': {'r4': 3, 'r5': 4, 'r6': 5, 'r7': 6, 'r1': 0, 'r2': 1, 'r3': 2, 'r8': 7, 'r9': 8}, 'crew_type': {'contractor': 4, 'civilian': 3, 'officer': 1, 'enlisted': 0, 'warrant_officer': 2}, 'skill_type': {'other': 1, 'maintenance': 0}}

{'availability_number': {'sra1': 0,
  'sra10': 9,
  'sra11': 10,
  'sra12': 11,
  'sra13': 12,
  'sra14': 13,
  'sra15': 14,
  'sra2': 1,
  'sra3': 2,
  'sra4': 3,
  'sra5': 4,
  'sra6': 5,
  'sra7': 6,
  'sra8': 7,
  'sra9': 8},
 'crew_type': {'civilian': 3,
  'contractor': 4,
  'enlisted': 0,
  'officer': 1,
  'warrant_officer': 2},
 'maintenance_level': {'depot': 2, 'intermediate': 1, 'onboard': 0},
 'maintenance_type': {'corrective': 1, 'preventive': 0},
 'other_availability_number': {'sra1': 0,
  'sra10': 9,
  'sra11': 10,
  'sra12': 11,
  'sra13': 12,
  'sra14': 13,
  'sra15': 14,
  'sra2': 1,
  'sra3': 2,
  'sra4': 3,
  'sra5': 4,
  'sra6': 5,
  'sra7': 6,
  'sra8': 7,
  'sra9': 8},
 'rank': {'r1': 0,
  'r2': 1,
  'r3': 2,
  'r4': 3,
  'r5': 4,
  'r6': 5,
  'r7': 6,
  'r8': 7,
  'r9': 8},
 'skill_type': {'maintenance': 0, 'other': 1},
 'support_type': {'admin': 1, 'logistics': 0, 'training': 2}}

In [4]:
model = pysd.load('tests/pdatvensim8k_RW_v2_4.py')

SyntaxError: unexpected character after line continuation character (pdatvensim8k_RW_v2_4.py, line 3081)

In [3]:
model.run()

IndexError: too many indices for array

In [7]:
a = model.run(flatten_subscripts=False)
a.head()

,stock_a
0,"[0.0, 0.0, 0.0]"
1,"[0.01, 0.2, 0.3]"
2,"[0.02, 0.4, 0.6]"
3,"[0.03, 0.6, 0.9]"
4,"[0.04, 0.8, 1.2]"


In [8]:
a = model.run(flatten_subscripts=True)
a.head()

,stock_a____entry_1,stock_a____entry_2,stock_a____entry_3
0,0.00,0.0,0.0
1,0.01,0.2,0.3
2,0.02,0.4,0.6
3,0.03,0.6,0.9
4,0.04,0.8,1.2


In [5]:
fmod(5,2)

1

In [4]:
model.components.stock_a.dimension_directory

{'one_dimensional_subscript': 0, 'second_dimension_subscript': 1}

In [26]:
a = pd.DataFrame(data=1, index=range(5), columns=['hi','there'])
a

,hi,there
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [33]:
a.columns = ['hi', 'mounir']

In [34]:
a

,hi,mounir
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [37]:
b = set(['hi', 'there'])
b.add('mounir')
b

{'hi', 'mounir', 'there'}

In [52]:
c = set(['hi'])
#len(c)
list(c)[0]

'hi'

In [49]:
shape = np.zeros(4)
shape

array([ 0.,  0.,  0.,  0.])

In [39]:


subscript_references = [['one_dimensional_subscript', 'second_dimension_subscript']]

num_dimensions = len(subscript_references[0])
reference_sets = [set() for _ in range(num_dimensions)]
    
for subscript_reference in subscript_references:
    [reference_sets[i].add(element) for i, element in enumerate(subscript_reference)]

print reference_sets

[set(['one_dimensional_subscript']), set(['second_dimension_subscript'])]


In [22]:
a.loc[0]['stock_a'].flatten()

array([ 1.,  2.,  2.,  4.,  3.,  5.,  4.,  3.,  5.,  1.,  6.,  4.])

In [ ]:
func(df, subdict, dimdict)

In [6]:
model.components._subscript_dict

{'one_dimensional_subscript': {'entry_1': 0, 'entry_2': 1, 'entry_3': 2},
 'second_dimension_subscript': {'column_1': 0, 'column_2': 1},
 'third_dimension_subscript': {'depth_1': 0, 'depth_2': 1}}

In [ ]:

#inputs
dataframe of model output

subscript_dict = {'one_dimensional_subscript': {'entry_1': 0, 'entry_2': 1, 'entry_3': 2},
 'second_dimension_subscript': {'column_1': 0, 'column_2': 1},
 'third_dimension_subscript': {'depth_1': 0, 'depth_2': 1}}

dimdict = {'one_dimensional_subscript': 0,
 'second_dimension_subscript': 1,
 'third_dimension_subscript': 2}

#output
#flattened dataframe of output with names akin to: 
    Rate A[Entry 1,Column 1]	Rate A[Entry 1,Column 2]	Rate A[Entry 2,Column 1]	Rate A[Entry 2,Column 2]	Rate A[Entry 3,Column 1]	Rate A[Entry 3,Column 2]

In [ ]:
list(['stock_a__entry_1__column_2__depth_2'])

In [14]:
subscript_dict = {'one_dimensional_subscript': (0, {'entry_1': 0, 'entry_2': 1, 'entry_3': 2}),
 'second_dimension_subscript': (1, {'column_1': 0, 'column_2': 1}),
 'third_dimension_subscript': (2, {'depth_1': 0, 'depth_2': 1})}

In [13]:
array = model.components.stock_a()
array

array([[[  1.99,   3.98],
        [  3.98,   8.95]],

       [[  5.97,   6.98],
        [  7.96,   6.96]],

       [[  9.95,   5.95],
        [ 11.94,   9.94]]])

In [15]:
for family, subscripts in subscript_dict.iteritems():
    

second_dimension_subscript
one_dimensional_subscript
third_dimension_subscript


In [12]:
array.flatten()

array([  1.99,   3.98,   3.98,   8.95,   5.97,   6.98,   7.96,   6.96,
         9.95,   5.95,  11.94,   9.94])

# prototype

In [ ]:
def get_array_info(subs, dictofsubs):
    """
    Returns information needed to create and access members of the numpy array
    based upon the string names given to them in the model file.

    Parameters
    ----------
    subs : Array of strings of subscripts

    returns
    -------
    A dictionary of the dimensions associating their names with their numpy indices
    directory = {'dimension name 1':0, 'dimension name 2':1}
    
    A list of the length of each dimension. Equivalently, the shape of the array:
    shape = [5,4]

    """
    
    for 
    
    # Todo: rework this to account for subarrays
    # todo: make this elementstr or something
    position=[]
    elements=element.replace('!','').replace('','').split(',')
    for element in elements:
        if element in dictofsubs.keys():
            position.append(len(dictofsubs[element]))
        else:
            for d in dictofsubs.itervalues():
                try:
                    (d[element])
                except: pass
                else:
                    position.append(len(d))

    return position



In [55]:
def hi:
    return 1

SyntaxError: invalid syntax (<ipython-input-55-ad431173dcc6>, line 1)

In [27]:
a = {'a':4, 'b':2, 'c':'r'}

a.keys()[a.values().index('r')]

'c'

In [29]:
[a.keys()[a.values().index(x)] for x in sorted(a.values())]


['b', 'a', 'c']

In [75]:
a.keys()

['a', 'c', 'b']

In [76]:
a.values()

[4, 'r', 2]

In [ ]:
result = df['stock_a'].apply(lambda x: pd.Series(x.flatten()))

In [77]:
a = 'hi there'
[a]

['hi there']

In [5]:
modelfile = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'

In [12]:
import os.path
directory = os.path.dirname(modelfile)
if os.path.exists(directory+'/output.csv'):
    canon = pd.read_csv(directory+'/output.csv', index_col='Time')
elif os.path.exists(directory+'/output.tab'):
    print directory+'/output.tab'
    canon = pd.read_table(directory+'/output.tab', index_col='Time')
else:
    raise IOError

tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/output.tab


In [41]:
import traceback

try:
    err=False
    print 'max'
    #raise IOError
except IOError:
    print 'hi'
    err = True
except AssertionError:
    print 'there'
    err = True
except:
    print 'bob'
    err = True
finally:
    if err:
        print 'smile'

max


In [13]:
canon

,"Inflow A[Entry 1,Column 1]","Inflow A[Entry 1,Column 2]","Inflow A[Entry 2,Column 1]","Inflow A[Entry 2,Column 2]","Inflow A[Entry 3,Column 1]","Inflow A[Entry 3,Column 2]","Initial Values[Entry 1,Column 1]","Initial Values[Entry 1,Column 2]","Initial Values[Entry 2,Column 1]","Initial Values[Entry 2,Column 2]",...,"Rate A[Entry 2,Column 1]","Rate A[Entry 2,Column 2]","Rate A[Entry 3,Column 1]","Rate A[Entry 3,Column 2]","Stock A[Entry 1,Column 1]","Stock A[Entry 1,Column 2]","Stock A[Entry 2,Column 1]","Stock A[Entry 2,Column 2]","Stock A[Entry 3,Column 1]","Stock A[Entry 3,Column 2]"
Time,,,,,,,,,,,,,,,,,,,,,
0,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.00,2.00,3.00000,4.00,5.00000,6.00
1,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.01,2.02,3.03000,4.04,5.05000,6.06
2,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.02,2.04,3.06000,4.08,5.10000,6.12
3,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.03,2.06,3.09000,4.12,5.15000,6.18
4,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.04,2.08,3.12000,4.16,5.20000,6.24
5,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.05,2.10,3.15000,4.20,5.25000,6.30
6,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.06,2.12,3.18000,4.24,5.30000,6.36
7,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.07,2.14,3.21000,4.28,5.35000,6.42
8,0.01,0.02,0.03,0.04,0.05,0.06,1,2,3,4,...,0.03,0.04,0.05,0.06,1.08,2.16,3.24000,4.32,5.40000,6.48
